* Étape 1 : Définir les données XOR


In [107]:
import pandas as pd
import re

def extract_data(input_path, pattern):
    with open(input_path, 'r') as file:
        content = file.read()
    matches = re.findall(pattern, content, re.MULTILINE)
    data = []
    for match in matches:
        numbers = match[1].replace('\n', ' ').split()
        numbers_float = [float(number) for number in numbers]
        data.append(numbers_float)
    return pd.DataFrame(data)

rocks_path = 'data/sonar.rocks'
mines_path = 'data/sonar.mines'


train_pattern = r'\*(CR|CM)\d+:\n\{([\d\s\.\n]+)\}'
test_pattern = r'^(?!\*)(CR|CM)\d+:\n\{([\d\s\.\n]+)\}'


rocks_train_df = extract_data(rocks_path, train_pattern)
mines_train_df = extract_data(mines_path, train_pattern)
rocks_train_df['Label'] = 'R'  
mines_train_df['Label'] = 'M' 
train_df = pd.concat([rocks_train_df, mines_train_df], ignore_index=True).sample(frac=1).reset_index(drop=True)

rocks_test_df = extract_data(rocks_path, test_pattern)
mines_test_df = extract_data(mines_path, test_pattern)
rocks_test_df['Label'] = 'R'  
mines_test_df['Label'] = 'M' 
test_df = pd.concat([rocks_test_df, mines_test_df], ignore_index=True).sample(frac=1).reset_index(drop=True)

Préparation des Données


In [108]:
# Assurons-nous que les étiquettes sont numériques
train_df['Label'] = train_df['Label'].map({'R': 0, 'M': 1})
test_df['Label'] = test_df['Label'].map({'R': 0, 'M': 1})

# Séparation des caractéristiques et des étiquettes
X_train = train_df.drop('Label', axis=1)
y_train = train_df['Label']
X_test = test_df.drop('Label', axis=1)
y_test = test_df['Label']
X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])



Minimerror Simplifié


In [110]:
import numpy as np
import matplotlib.pyplot as plt

def predict_perceptron(inputs, weights):
    activation = np.dot(inputs, weights[1:]) + weights[0]
    return 1 if activation >= 0 else -1

# Mise à jour des poids selon Minimerror
def update_weights_minimerror(X, y, weights, T, learning_rate):
    weights_norm = np.linalg.norm(weights)
    gradients = np.zeros_like(weights)
    
    for mu in range(X.shape[0]):
        xi = X[mu, :]
        tau = y[mu]
        gamma = (tau * np.dot(xi, weights)) / weights_norm
        tanh_gamma = np.tanh(gamma / (2 * T))
        factor = (1 - tanh_gamma**2) / (2 * T)
        gradient_mu = factor * (xi * tau / weights_norm - weights * (tau * np.dot(weights, xi)) / weights_norm**3)
        gradients += gradient_mu
    
    new_weights = weights - learning_rate * gradients / X.shape[0]
    return new_weights



# Calcul de l'erreur (MSE)
def compute_error(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

# Stabilité des exemples de test
def compute_stabilities(X, weights):
    # La normalisation des poids devrait inclure tous les poids, y compris le biais
    norm_weights = np.linalg.norm(weights)
    # Utiliser tous les poids pour calculer la distance à l'hyperplan
    return np.dot(X, weights) / norm_weights


# Initialisation des variables
np.random.seed(42)
weights = np.random.normal(0, 1, X_train.shape[1])
learning_rate = 0.01
epochs = 1

# Initialisation de beta
initial_beta = 1
final_beta = 10  # Supposons que nous voulons que beta atteigne 10 à la fin de l'entraînement
beta_increment = (final_beta - initial_beta) / epochs

Ea = 1
Eg = 1
# Boucle d'apprentissage avec ajustement de beta
while Ea > 0.2 and Eg > 0.2:
    beta = initial_beta + epoch * beta_increment
    T=1/beta
    weights = update_weights_minimerror(X_train, y_train, weights, T, learning_rate)
    # Evaluation
    y_pred_train = predict(X_train, weights) > 0.5
    y_pred_test = predict(X_test, weights) > 0.5
    Ea = compute_error(y_train, y_pred_train)
    Eg = compute_error(y_test, y_pred_test)
    epoch += 1
    
print(epoch)


print(f"Erreur d'apprentissage (Ea): {Ea}")
print(f"Erreur de généralisation (Eg): {Eg}")
print("Poids du perceptron (W):", weights)

# Calcul et affichage des stabilités pour l'ensemble de test
stabilities = compute_stabilities(X_test, weights)
plt.hist(stabilities, bins=20)
plt.title('Distribution des Stabilités des Exemples de Test')
plt.xlabel('Stabilité')
plt.ylabel('Nombre d\'Exemples')
plt.show()


ValueError: shapes (61,) and (62,) not aligned: 61 (dim 0) != 62 (dim 0)